<a href="https://colab.research.google.com/github/dianakang/DIANA_TimeSeries/blob/master/%5B%EB%94%A5%EB%9F%AC%EB%8B%9D%EC%9C%BC%EB%A1%9C_%EA%B1%B7%EB%8A%94_%EC%8B%9C%EA%B3%84%EC%97%B4_%EC%98%88%EC%B8%A1%5D_Ch4_%ED%9A%8C%EA%B7%80_%EB%AA%A8%EB%8D%B8%EC%9D%98_%ED%8C%90%EB%B3%84%EC%8B%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 목차

*   4.1. 회귀 모델의 판별식
*   4.2. 회귀 모델 코드 작성하기
  *   4.2.1. Validation 추가
  *   4.2.2. 데이터 분리
      *   (1) 테스트 데이터로 예측하는 방법
      *   (2) 새로운 데이터로 예측하는 방법
  *   4.2.3. train_test_split
*   4.3. 함수형 모델
  *   4.3.1. 1:1 모델
  *   4.3.2. 다:다 모델
  *   4.3.3. 다:1 모델
  *   4.3.4. 1:다 모델


---

# 4. 회귀 모델의 판별식

## 4.1. 회귀 모델의 판별식

**RMSE(평균 제곱근 오차, Root Mean Squared Error)는 회귀 분석에서 모델을 평가할 때 가장 많이 쓰는 지표 중 하나**이다.

대부분의 모든 파이썬과 딥러닝, 머신러닝은 API나 프레임워크, 함수가 거의 만들어져 있다.

하지만 아쉽게도 RMSE는 아직 준비가 안되어 있는 듯 하다. 그래서 간단하게 함수를 만들어 사용한다.

MSE에 루트를 씌운 것이 RMSE이다. 즉, RMSE는 원래 데이터에서 평균을 뺀 값을 제곱하여 모두 더한 뒤 전체 개수로 나눈 값에 루트를 씌운 것이다. 

**RMSE가 낮을수록 정밀도가 높다.**

In [3]:
from keras.models import Sequential
from keras.layers import Dense
import numpy as np 

x_train = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
y_train = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

x_test = np.array([101, 102, 103, 104, 105, 106, 107, 108, 109, 110])
y_test = np.array([101, 102, 103, 104, 105, 106, 107, 108, 109, 110])

model = Sequential()
model.add(Dense(5, input_dim=1, activation='relu'))
model.add(Dense(3))
model.add(Dense(1, activation='relu'))

model.summary()

model.compile(loss='mse', optimizer='adam', metrics=['mse'])

model.fit(x_train, y_train, epochs=300, batch_size=1, validation_data=(x_test, y_test))

loss, acc = model.evaluate(x_test, y_test, batch_size=1)

print('loss : ', loss)
y_predict = model.predict(x_test)
print('결과물 : \n', y_predict)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 5)                 10        
_________________________________________________________________
dense_4 (Dense)              (None, 3)                 18        
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 4         
Total params: 32
Trainable params: 32
Non-trainable params: 0
_________________________________________________________________
Epoch 1/300
10/10 [==============================] - 0s 16ms/step - loss: 21.2227 - mse: 21.2227 - val_loss: 8010.7642 - val_mse: 8010.7642
Epoch 2/300
10/10 [==============================] - 0s 5ms/step - loss: 27.8609 - mse: 27.8609 - val_loss: 7617.9468 - val_mse: 7617.9468
Epoch 3/300
10/10 [==============================] - 0s 6ms/step - loss: 21.5738 - mse: 21.5738 - val_loss: 7

In [11]:
# RMSE 구하기 
from sklearn.metrics import mean_squared_error

def RMSE(y_test, y_predict):
    return np.sqrt(mean_squared_error(y_test, y_predict))

print('RMSE : ', RMSE(y_test ,y_predict))

RMSE :  0.16617615553804171


이어서 RMSE와 함께 회귀 분석에서 가장 많이 쓰는 지표인 R2를 구해보자.

R2는 여러 가지로 불린다. R2, R2 Score, R제곱, 설명력, 결정계수 등이 있지만 보통 R2 또는 결정계수로 많이 불린다. 

RMSE와 반대로 높을수록 좋은 지표이며, 0-1 사이의 수치가 출력된다. 

만약 0,73의 값이 나온다면, 내가 만든 모델식이 R2로 계산했을 떄 73%의 설명력을 가진다고 해석할 수 있다.

R2는 사이킷런에서 함수 형태로 제공해주기 때문에 사이킷런을 import해서 그대로 사용하면 된다.

In [12]:
# R2 구하기
from sklearn.metrics import r2_score
r2_y_predict = r2_score(y_test, y_predict)
print("R2 : ", r2_y_predict)

R2 :  0.9966527861006784


## 4.2. 회귀 모델 코드 작성하기

### 4.2.1. Validation 추가

앞에서 모델을 훈련시킬 때(fit) 검증값을 test로 하였다. 그러나 훈련셋에 검증값이 들어가고 그 검증값으로 다시 테스트를 한다는 것은 평가에 검증값이 반영되는 문제가 있다. 

따라서 **훈련셋, 테스트셋, 검증셋은 엄밀히 분리가 되는 것이 좋은 데이터의 형태이다.**

**일반적으로 Train 데이터의 일부를 잘라서 Validation 데이터로 사용하는 것이 좋다.**

In [13]:
# 1. 데이터 준비
import numpy as np
x_train = np.array([1,2,3,3,4,5,6,7,8,9,10])
y_train = np.array([1,2,3,3,4,5,6,7,8,9,10])

x_test = np.array([11,12,13,14,15,16,17,18,19,20])
y_test = np.array([11,12,13,14,15,16,17,18,19,20])

x_val = np.array([101,102,103,104,105])
y_val = np.array([101,102,103,104,105])


# 2. 모델 구성
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
model.add(Dense(5, input_shape=(1,), activation='relu'))
model.add(Dense(3))
model.add(Dense(4))
model.add(Dense(1))

# 3. 모델 훈련
model.compile(optimizer='adam', loss='mse', metrics=['mse'])
model.fit(x_train, y_train, epochs=500, batch_size=1, validation_data=(x_val, y_val))

# 4. 평가예측
mse = model.evaluate(x_test, y_test, batch_size=1)
print('mse : ', mse)

y_predict = model.predict(x_test)
print(y_predict)

Epoch 1/500
11/11 [==============================] - 0s 16ms/step - loss: 25.2323 - mse: 25.2323 - val_loss: 12458.6465 - val_mse: 12458.6465
Epoch 2/500
11/11 [==============================] - 0s 4ms/step - loss: 51.6139 - mse: 51.6139 - val_loss: 11569.2480 - val_mse: 11569.2480
Epoch 3/500
11/11 [==============================] - 0s 4ms/step - loss: 27.4380 - mse: 27.4380 - val_loss: 10832.1621 - val_mse: 10832.1621
Epoch 4/500
11/11 [==============================] - 0s 4ms/step - loss: 38.3012 - mse: 38.3012 - val_loss: 10039.5391 - val_mse: 10039.5391
Epoch 5/500
11/11 [==============================] - 0s 4ms/step - loss: 23.8661 - mse: 23.8661 - val_loss: 9342.2812 - val_mse: 9342.2812
Epoch 6/500
11/11 [==============================] - 0s 4ms/step - loss: 30.6957 - mse: 30.6957 - val_loss: 8543.5137 - val_mse: 8543.5137
Epoch 7/500
11/11 [==============================] - 0s 4ms/step - loss: 21.2914 - mse: 21.2914 - val_loss: 7798.6982 - val_mse: 7798.6982
Epoch 8/500
11/11 

1에서 10까지의 데이터를 훈련시킬 때 101에서 105의 데이터를 검증용 데이터로 사용한다. 이후 11에서 20까지의 데이터로 테스트한다.

mse는 5.36601882719645e-12로 아주 낮은 수치가 나왔고, predict도 거의 정확하게 나왔다. 

이제 여기에 RMSE와 R2를 추가해보자.

In [14]:
# RMSE 구하기
from sklearn.metrics import mean_squared_error
def RMSE(y_test, y_predict):
  return np.sqrt(mean_squared_error(y_test, y_predict))
print("RMSE : ", RMSE(y_test, y_predict))

# R2 구하기
from sklearn.metrics import r2_score
r2_y_predict = r2_score(y_test, y_predict)
print("R2 : ", r2_y_predict)

RMSE :  2.6117446780451113e-06
R2 :  0.9999999999991732


-> RMSE는 낮을수록, R2는 1에 가까울수록 정확한 예측치가 된다. 

---


 

### 4.2.2 데이터 분리

#### (1) 테스트 데이터로 예측하는 방법

이번에는 데이터를 일일이 쓰지 않고, 1부터 100까지의 정수형 데이터셋을 잘라서 사용해보겠다. 

In [15]:
# 1. 데이터 준비
x = np.array(range(1,101))
y = np.array(range(1,101))

데이터셋이 준비되었으니 이제 이것을 6:2:2의 비율(Train:Val:Test)로 나눠보겠다.

In [16]:
# 2. Train:Val:Test로 나누기
x_train = x[:60]
x_val = x[60:80]
x_test = x[80:]

y_train = y[:60]
y_val = y[60:80]
y_test = y[80:]

In [17]:
# 3. 모델 구성
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
model.add(Dense(5, input_shape=(1,), activation='relu'))
model.add(Dense(3))
model.add(Dense(4))
model.add(Dense(1))

# 4. 모델 훈련
model.compile(optimizer='adam', loss='mse', metrics=['mse'])
model.fit(x_train, y_train, epochs=500, batch_size=1, validation_data=(x_val, y_val))

# 5. 평가예측
mse = model.evaluate(x_test, y_test, batch_size=1)
print('mse : ', mse)

y_predict = model.predict(x_test)
print(y_predict)

# 6. RMSE 구하기
from sklearn.metrics import mean_squared_error
def RMSE(y_test, y_predict):
  return np.sqrt(mean_squared_error(y_test, y_predict))
print("RMSE : ", RMSE(y_test, y_predict))

# 7. R2 구하기
from sklearn.metrics import r2_score
r2_y_predict = r2_score(y_test, y_predict)
print("R2 : ", r2_y_predict)

Epoch 1/500
60/60 [==============================] - 1s 4ms/step - loss: 1627.5283 - mse: 1627.5283 - val_loss: 5772.1870 - val_mse: 5772.1870
Epoch 2/500
60/60 [==============================] - 0s 2ms/step - loss: 1381.8638 - mse: 1381.8638 - val_loss: 4661.1470 - val_mse: 4661.1470
Epoch 3/500
60/60 [==============================] - 0s 2ms/step - loss: 1071.1294 - mse: 1071.1294 - val_loss: 3206.3130 - val_mse: 3206.3130
Epoch 4/500
60/60 [==============================] - 0s 2ms/step - loss: 699.7870 - mse: 699.7870 - val_loss: 1691.9359 - val_mse: 1691.9359
Epoch 5/500
60/60 [==============================] - 0s 2ms/step - loss: 284.3376 - mse: 284.3376 - val_loss: 456.1407 - val_mse: 456.1407
Epoch 6/500
60/60 [==============================] - 0s 2ms/step - loss: 58.2506 - mse: 58.2506 - val_loss: 51.9423 - val_mse: 51.9423
Epoch 7/500
60/60 [==============================] - 0s 2ms/step - loss: 5.2156 - mse: 5.2156 - val_loss: 4.2126 - val_mse: 4.2126
Epoch 8/500
60/60 [======

-> mse, RMSE 모두 아주 낮은 수치가 나왔고 R2 역시 0.999...로 1에 근접하는 아주 좋은 평가가 나왔다. 

  예측값도 81에서 100에 근접한 값으로 입력한 값(x_test)과 거의 근사치로 출력되었다. 

  하지만 이 모델에도 약간의 문제가 있는데, 그것은 바로 x_test 값으로 predict를 한 것이다. 

  가급적 **test한 값보다는 새로운 데이터로 예측하는 것이 좋기 때문에**, x_predict를 새로 입력하여 예측해보도록 하겠다. 

  x_predict는 x의 범위 바깥쪽의 새로운 데이터로 하겠다.

#### (2) 새로운 데이터로 예측하는 방법

In [18]:
# 1. 데이터 준비
x = np.array(range(1,101))
y = np.array(range(1,101))


# 2. 6:2:2비율로 Train:Val:Test로 나누기
x_train = x[:60]
x_val = x[60:80]
x_test = x[80:]

y_train = y[:60]
y_val = y[60:80]
y_test = y[80:]

x_predict = np.array(range(101,111))   ## 추가


# 3. 모델 구성
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
model.add(Dense(5, input_shape=(1,), activation='relu'))
model.add(Dense(3))
model.add(Dense(4))
model.add(Dense(1))

# 4. 모델 훈련
model.compile(optimizer='adam', loss='mse', metrics=['mse'])
model.fit(x_train, y_train, epochs=500, batch_size=1, validation_data=(x_val, y_val))

# 5. 평가예측
mse = model.evaluate(x_test, y_test, batch_size=1)
print('mse : ', mse)

y_predict = model.predict(x_predict)   ## x_test > x_predict로 수정
print(y_predict)


Epoch 1/500
60/60 [==============================] - 1s 4ms/step - loss: 602.9449 - mse: 602.9449 - val_loss: 736.3043 - val_mse: 736.3043
Epoch 2/500
60/60 [==============================] - 0s 2ms/step - loss: 141.0249 - mse: 141.0249 - val_loss: 42.5977 - val_mse: 42.5977
Epoch 3/500
60/60 [==============================] - 0s 2ms/step - loss: 3.3076 - mse: 3.3076 - val_loss: 0.4176 - val_mse: 0.4176
Epoch 4/500
60/60 [==============================] - 0s 2ms/step - loss: 0.1487 - mse: 0.1487 - val_loss: 0.3340 - val_mse: 0.3340
Epoch 5/500
60/60 [==============================] - 0s 2ms/step - loss: 0.1170 - mse: 0.1170 - val_loss: 0.3841 - val_mse: 0.3841
Epoch 6/500
60/60 [==============================] - 0s 2ms/step - loss: 0.1231 - mse: 0.1231 - val_loss: 0.2749 - val_mse: 0.2749
Epoch 7/500
60/60 [==============================] - 0s 2ms/step - loss: 0.1005 - mse: 0.1005 - val_loss: 0.3579 - val_mse: 0.3579
Epoch 8/500
60/60 [==============================] - 0s 2ms/step - lo

101에서 110까지의 값을 x로 주고 예측을 시켰을 때의 결과이다.

train과 test에서 제공된 데이터가 아닌 전혀 다른 데이터로 predict을 시켰으나 거의 정확하게 예측되었다!

### 4.2.3. train_test_split

지금까지는 데이터를 손으로 잘라서 사용했다. 사이킷런에서 이미 잘 구현된 train_test_split 함수를 제공하니 이것을 사용해보자.

In [19]:
# sklean의 train_test_split 함수 사용
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=66, test_size=0.4, shuffle=False)

x, y에 데이터의 x값과 y값을 넣었다. test_size=0.4는 test_size가 40%라는 의미이다. 이는 train_size=0.6이라고도 할 수 있다. 

shuffle이라는 파라미터가 추가되었다는 점도 눈여겨 봐야한다. 잘라낸 데이터를 섞을 것이냐는 의미이다. 이때 x와 y의 쌍까지 섞이지는 않는다. 

---

train_test_split는 인잣값으로 x input 데이터와 y output 데이터를 받는다. 옵션값으로는 test_size, train_size, random_state, shuffle등이 있다.

*   **test_size** : float로 받으며 0.0~1.0까지 입력한다. 정수를 입력하면 데이터의 절대 수를 의미한다. 옵션값이 지정되지 않았을 경우 train_size에 따라 결정되는데, train_size도 None 이면 0.25로 맞춰진다.

*   **train_size** : float 로 받으며 0.0~1.0까지 입력한다. 정수를 입력하면 데이터의 절대 수를 의미한다. 옵션값이 지정되지 않았을 경우 test_size 따라 자동 결정된다.

*   **random_state** : 재현가능(for reproducibility)하도록 난수의 초기값을 설정해주는 것이다.

*   **shuffle** : 분할하기 전에 데이터를 섞을 것인지를 의미한다. 기본값은 True이다.

In [20]:
# 1. 데이터 준비 
import numpy as np
x = np.array(range(1, 101))
y = np.array(range(1, 101))

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.4, random_state=66, shuffle=False)
x_val, x_test, y_val, y_test = train_test_split(x_test, y_test, test_size=0.5, random_state=66, shuffle=False)
## test 데이터의 50%를 validation에 배분하게 되어 train:val:test의 비율이 6:2:2가 된다.


# 2. 모델 구성
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
model.add(Dense(5, input_shape=(1,), activation='relu'))
model.add(Dense(4))
model.add(Dense(3))
model.add(Dense(1))

# 3. 훈련
model.compile(optimizer='adam', loss='mse', metrics=['mse'])
model.fit(x_train, y_train, batch_size=1, epochs=300, validation_data=(x_val, y_val))

# 4. 평가 예측
mse = model.evaluate(x_test, y_test, batch_size=1)
print('mse :', mse)

y_predict = model.predict(x_test)
print(y_predict)

Epoch 1/300
60/60 [==============================] - 1s 4ms/step - loss: 1486.2996 - mse: 1486.2996 - val_loss: 3176.3684 - val_mse: 3176.3684
Epoch 2/300
60/60 [==============================] - 0s 2ms/step - loss: 593.3036 - mse: 593.3036 - val_loss: 1502.8042 - val_mse: 1502.8042
Epoch 3/300
60/60 [==============================] - 0s 2ms/step - loss: 308.2341 - mse: 308.2341 - val_loss: 335.0134 - val_mse: 335.0134
Epoch 4/300
60/60 [==============================] - 0s 2ms/step - loss: 47.1245 - mse: 47.1245 - val_loss: 26.1325 - val_mse: 26.1325
Epoch 5/300
60/60 [==============================] - 0s 2ms/step - loss: 2.5936 - mse: 2.5936 - val_loss: 1.7608 - val_mse: 1.7608
Epoch 6/300
60/60 [==============================] - 0s 2ms/step - loss: 0.3555 - mse: 0.3555 - val_loss: 0.7319 - val_mse: 0.7319
Epoch 7/300
60/60 [==============================] - 0s 2ms/step - loss: 0.2412 - mse: 0.2412 - val_loss: 0.7754 - val_mse: 0.7754
Epoch 8/300
60/60 [==============================

In [23]:
#RMSE 구하기
from sklearn.metrics import mean_squared_error
def RMSE(y_test, y_predict):
    return np.sqrt(mean_squared_error(y_test, y_predict))
print("RMSE : ", RMSE(y_test, y_predict))

# R2 구하기
from sklearn.metrics import r2_score
r2_y_predict = r2_score(y_test, y_predict)
print("R2 : ", r2_y_predict)

RMSE :  0.00026405311326548856
R2 :  0.9999999979030362


## 4.3. 함수형 모델

### 4.3.1. 1:1 모델

지금까지 모델을 만들 때 model = Sequential()을 이용하여 순차적 모델만을 만들어왔다. 케라스 딥러닝에서는 두 가지 모델을 구성하는 방식이 있다. 

기존에 배웠던 순차적 모델과 이번에 배울 함수형 모델이다.

모델의 간결함은 순차적 모델이 우수하나, 앞으로 모델이 길어지고 앙상블 등의 여러 가지 기법을 사용하고자 하면 함수형 모델은 필수이다.

In [25]:
# 1. 데이터 준비 
import numpy as np
x = np.array(range(1, 101))
y = np.array(range(1, 101))

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.4, random_state=66, shuffle=False)
x_val, x_test, y_val, y_test = train_test_split(x_test, y_test, test_size=0.5, random_state=66, shuffle=False)
## test 데이터의 50%를 validation에 배분하게 되어 train:val:test의 비율이 6:2:2가 된다.


# 2. 모델 구성
from keras.models import Sequential, Model   ## Model을 추가해준다.
from keras.layers import Dense, Input  ## Input 레이어를 추가해준다.

input1 = Input(shape=(1,))
dense1 = Dense(5, activation='relu')(input1)
dense2 = Dense(3)(dense1)
dense3 = Dense(4)(dense2)
output1 = Dense(1)(dense3)

model = Model(inputs = input1, outputs= output1)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
dense_26 (Dense)             (None, 5)                 10        
_________________________________________________________________
dense_27 (Dense)             (None, 3)                 18        
_________________________________________________________________
dense_28 (Dense)             (None, 4)                 16        
_________________________________________________________________
dense_29 (Dense)             (None, 1)                 5         
Total params: 49
Trainable params: 49
Non-trainable params: 0
_________________________________________________________________


-> 총 49개의 파라미터를 사용한 간단한 심층 신경망 모델이 잘 구현되었음을 확인할 수 있다. 

*   Input(shape=(1,))

  : Input레이어를 이용하여 shape을 구성한다. 1개의 컬럼이 들어가므로 shape에 1을 입력한다.

*   다음 레이어들부터는 상위층에서 출력된 레이어의 이름을 하위층의 가장 끝부분에 명시해준다.

*   마지막으로 Model을 통해 input 레이어와 output 레이어를 엮어준다.



In [26]:
# 3. 훈련
model.compile(optimizer='adam', loss='mse', metrics=['mse'])
model.fit(x_train, y_train, batch_size=1, epochs=100, validation_data=(x_val, y_val))

# 4. 평가 예측
mse = model.evaluate(x_test, y_test, batch_size=3)
print('mse :', mse)

y_predict = model.predict(x_test)
print(y_predict)

#RMSE 구하기
from sklearn.metrics import mean_squared_error
def RMSE(y_test, y_predict):
    return np.sqrt(mean_squared_error(y_test, y_predict))
print("RMSE : ", RMSE(y_test, y_predict))

# R2 구하기
from sklearn.metrics import r2_score
r2_y_predict = r2_score(y_test, y_predict)
print("R2 : ", r2_y_predict)

Epoch 1/100
60/60 [==============================] - 1s 4ms/step - loss: 1085.6285 - mse: 1085.6285 - val_loss: 3990.5090 - val_mse: 3990.5090
Epoch 2/100
60/60 [==============================] - 0s 2ms/step - loss: 936.7070 - mse: 936.7070 - val_loss: 2741.6665 - val_mse: 2741.6665
Epoch 3/100
60/60 [==============================] - 0s 2ms/step - loss: 540.1752 - mse: 540.1752 - val_loss: 1542.9612 - val_mse: 1542.9612
Epoch 4/100
60/60 [==============================] - 0s 2ms/step - loss: 294.1379 - mse: 294.1379 - val_loss: 487.9391 - val_mse: 487.9391
Epoch 5/100
60/60 [==============================] - 0s 2ms/step - loss: 69.9544 - mse: 69.9544 - val_loss: 86.0607 - val_mse: 86.0607
Epoch 6/100
60/60 [==============================] - 0s 2ms/step - loss: 7.9289 - mse: 7.9289 - val_loss: 9.8115 - val_mse: 9.8115
Epoch 7/100
60/60 [==============================] - 0s 2ms/step - loss: 1.4657 - mse: 1.4657 - val_loss: 3.6859 - val_mse: 3.6859
Epoch 8/100
60/60 [====================

-> mse, RMSE, R2 모두 괜찮은 값이 나왔고, 모델도 잘 돌아갔음을 확인할 수 있다.

### 4.3.2. 다:다 모델

지금까지는 입력되는 칼럼의 수가 1개였다(input_dim=1 또는 input_shape=(1,)). 이제부터 2개 이상의 컬럼이 입력되는 경우를 살펴보자. 

x와 y는 1에서 100까지의 정수와 301에서 400까지의 정수를 사용한다. 모델에 입력하기 위해서는 행과 열이 맞아야 한다. 

DNN의 구조에서는 차원(dimension)이 가장 중요하여, 행은 무시되고 열(컬럼)이 우선된다. 

그러다 보니 모델에서 입력할 때 input_dim = 1 또는 input_shape=(1,)과 같은 파라미터를 요구하게 된다. 

In [27]:
# 1. 데이터 준비 
import numpy as np
x = np.array([range(100), range(301, 401)])
y = np.array([range(100), range(301, 401)])

print(x.shape)
print(y.shape)

(2, 100)
(2, 100)


-> 위의 shape을 통해 본 결과를 확인해보면, 데이터 구조가 2행 100열로 컬럼이 100개이다.

우리가 원하는 구조는 100행 2열이므로, 행과 열을 바꿔줘야한다. 이때 필요한 메서드는 transpose()이다.

In [28]:
# 행 열 바꾸기 -> transpose() 이용
x = np.transpose(x)
y = np.transpose(y)

print(x.shape)
print(y.shape)

(100, 2)
(100, 2)


이제 우리가 원하는 100행 2열로 바뀌었다. input_dim = 2로 가능하게 데이터의 구조가 변경되었다. 

이제 데이터를 train, test, val로 분리한 후 모델을 구성해보자. 마찬가지로 6:2:2로 구성해볼 것이다. 

In [29]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=66,  test_size=0.4, shuffle=False)
x_val, x_test, y_val, y_test = train_test_split(x_test, y_test, random_state=66, test_size=0.5, shuffle=False)

# 2. 모델 구성
from keras.models import Sequential
from keras.layers import Dense
model = Sequential()

model.add(Dense(5, input_shape = (2, ), activation='relu'))  ## input_shape = (1, ) > input_shape = (2, )
model.add(Dense(3))
model.add(Dense(4))
model.add(Dense(2))

# 3. 훈련
model.compile(loss='mse', optimizer='adam', metrics=['mse'])
model.fit(x_train, y_train, epochs=300, batch_size=1, validation_data=(x_val, y_val))

# 4. 평가 예측
mse = model.evaluate(x_test, y_test, batch_size=1)
print('mse :', mse)

y_predict = model.predict(x_test)
print(y_predict)

#RMSE 구하기
from sklearn.metrics import mean_squared_error
def RMSE(y_test, y_predict):
    return np.sqrt(mean_squared_error(y_test, y_predict))
print("RMSE : ", RMSE(y_test, y_predict))

# R2 구하기
from sklearn.metrics import r2_score
r2_y_predict = r2_score(y_test, y_predict)
print("R2 : ", r2_y_predict)

Epoch 1/300
60/60 [==============================] - 1s 4ms/step - loss: 55416.9916 - mse: 55416.9916 - val_loss: 56108.6562 - val_mse: 56108.6562
Epoch 2/300
60/60 [==============================] - 0s 3ms/step - loss: 39419.7544 - mse: 39419.7544 - val_loss: 24917.3223 - val_mse: 24917.3223
Epoch 3/300
60/60 [==============================] - 0s 2ms/step - loss: 13861.5386 - mse: 13861.5386 - val_loss: 3551.2949 - val_mse: 3551.2949
Epoch 4/300
60/60 [==============================] - 0s 2ms/step - loss: 1039.6644 - mse: 1039.6644 - val_loss: 1031.8201 - val_mse: 1031.8201
Epoch 5/300
60/60 [==============================] - 0s 2ms/step - loss: 138.4219 - mse: 138.4219 - val_loss: 1109.6838 - val_mse: 1109.6838
Epoch 6/300
60/60 [==============================] - 0s 2ms/step - loss: 213.4631 - mse: 213.4631 - val_loss: 786.4839 - val_mse: 786.4839
Epoch 7/300
60/60 [==============================] - 0s 2ms/step - loss: 185.9861 - mse: 185.9861 - val_loss: 980.7390 - val_mse: 980.7390

### 4.3.3. 다:1 모델

다:1 모델은 다:다 모델에서 아웃풋만 1개인 경우를 말한다.

100개의 데이터씩 2개의 컬럼이 입력되어, 100개의 데이터 1개가 출력되는 구조이다. 위의 코드와 동일하지만, 최종 아웃풋 레이어만 1로 변경했다.

In [32]:
#1. 데이터 준비
import numpy as np

x = np.array([range(100), range(301,401)])
y = np.array(range(201,301))
x = np.transpose(x)

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=66, test_size=0.4, shuffle=False)
x_val, x_test, y_val, y_test = train_test_split(x_test, y_test, random_state=66, test_size=0.5, shuffle=False
                                                )
#2. 모델 구성
from keras.models import Sequential
from keras.layers import Dense
model = Sequential()
model.add(Dense(5, input_shape = (2, ), activation ='relu'))
model.add(Dense(3))
model.add(Dense(4))
model.add(Dense(1))  ## 최종 아웃풋 레이어 : 2 > 1로 변경

#3. 훈련
model.compile(loss='mse', optimizer='adam', metrics=['mse'])
model.fit(x_train, y_train, epochs=100, batch_size=1, validation_data=(x_val, y_val))

#4. 평가 예측
mse = model.evaluate(x_test, y_test, batch_size=1)
print("mse : ", mse)
y_predict = model.predict(x_test)

# RMSE 구하기
from sklearn.metrics import mean_squared_error
def RMSE(y_test, y_predict):
    return np.sqrt(mean_squared_error(y_test, y_predict))
print("RMSE : ", RMSE(y_test, y_predict))

# R2 구하기
from sklearn.metrics import r2_score
r2_y_predict = r2_score(y_test, y_predict)
print("R2 : ", r2_y_predict)

Epoch 1/100
60/60 [==============================] - 1s 4ms/step - loss: 57491.3963 - mse: 57491.3963 - val_loss: 39268.9570 - val_mse: 39268.9570
Epoch 2/100
60/60 [==============================] - 0s 3ms/step - loss: 27237.0625 - mse: 27237.0625 - val_loss: 12515.5176 - val_mse: 12515.5176
Epoch 3/100
60/60 [==============================] - 0s 3ms/step - loss: 6714.7126 - mse: 6714.7126 - val_loss: 621.5402 - val_mse: 621.5402
Epoch 4/100
60/60 [==============================] - 0s 2ms/step - loss: 260.0728 - mse: 260.0728 - val_loss: 2.3995 - val_mse: 2.3995
Epoch 5/100
60/60 [==============================] - 0s 3ms/step - loss: 0.3677 - mse: 0.3677 - val_loss: 4.9123 - val_mse: 4.9123
Epoch 6/100
60/60 [==============================] - 0s 2ms/step - loss: 0.1838 - mse: 0.1838 - val_loss: 4.7279 - val_mse: 4.7279
Epoch 7/100
60/60 [==============================] - 0s 2ms/step - loss: 0.1459 - mse: 0.1459 - val_loss: 4.3020 - val_mse: 4.3020
Epoch 8/100
60/60 [==================

### 4.3.4. 1:다 모델

In [33]:
#1. 데이터 준비하기
import numpy as np
x = np.array([range(100)])  
y = np.array([range(201,301), range(301,401)])   
x = np.transpose(x)
y = np.transpose(y)

print(x.shape)   ##  x는 (100,1)의 shape을 가지고 있는 데이터
print(y.shape)   ##  y는 (100,2)의 shape을 가지고 있는 데이터

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=66, test_size=0.4, shuffle=False)
x_val, x_test, y_val, y_test = train_test_split(x_test, y_test, random_state=66, test_size=0.5, shuffle=False)

print(x_test.shape)

#2. 모델 구성
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
model.add(Dense(5, input_shape = (1, ), activation ='relu'))   ## 입력 모델의 input_shape이 1로 바뀜.
model.add(Dense(3))
model.add(Dense(4))
model.add(Dense(2))    ## 마지막 출력의 Dense 층의 아웃풋을 2로 변경

#3. 훈련
model.compile(loss='mse', optimizer='adam', metrics=['mse'])
model.fit(x_train, y_train, epochs=100, batch_size=1, validation_data=(x_val, y_val))

#4. 평가 예측
loss, mse = model.evaluate(x_test, y_test, batch_size=1)
print("mse : ", mse)
y_predict = model.predict(x_test)
print(y_predict)

#RMSE 구하기
from sklearn.metrics import mean_squared_error
def RMSE(y_test, y_predict):
    return np.sqrt(mean_squared_error(y_test, y_predict))
print("RMSE : ", RMSE(y_test, y_predict))

# R2 구하기
from sklearn.metrics import r2_score
r2_y_predict = r2_score(y_test, y_predict)
print("R2 : ", r2_y_predict)

(100, 1)
(100, 2)
(20, 1)
Epoch 1/100
60/60 [==============================] - 1s 4ms/step - loss: 83915.6570 - mse: 83915.6570 - val_loss: 93482.6875 - val_mse: 93482.6875
Epoch 2/100
60/60 [==============================] - 0s 2ms/step - loss: 76254.4452 - mse: 76254.4452 - val_loss: 77054.6094 - val_mse: 77054.6094
Epoch 3/100
60/60 [==============================] - 0s 2ms/step - loss: 67867.0191 - mse: 67867.0191 - val_loss: 55395.3867 - val_mse: 55395.3867
Epoch 4/100
60/60 [==============================] - 0s 2ms/step - loss: 57627.4787 - mse: 57627.4787 - val_loss: 27212.3184 - val_mse: 27212.3184
Epoch 5/100
60/60 [==============================] - 0s 2ms/step - loss: 42418.8048 - mse: 42418.8048 - val_loss: 4654.7690 - val_mse: 4654.7690
Epoch 6/100
60/60 [==============================] - 0s 2ms/step - loss: 31484.6177 - mse: 31484.6177 - val_loss: 5403.5854 - val_mse: 5403.5854
Epoch 7/100
60/60 [==============================] - 0s 2ms/step - loss: 21294.3988 - mse: 21294

predict한 부분의 값이 엉망이다. 우리가 원하는 값이 나오지 않았다. RMSE도 높은 수치이고, R2는 아예 음수이다. 

결과값이 좋지는 않지만 이런 모델까지도 만들 수 있다는 것만 이해하면 된다!